# Prompt Caching

In [18]:
from anthropic import Anthropic

In [19]:
# Load environment variables
from helper import load_env
load_env()

In [20]:
client = Anthropic()
MODEL_NAME="claude-3-5-sonnet-20241022"

### Loading The Book

In [21]:
with open('files/frankenstein.txt', 'r') as file:
    book_content = file.read()

In [22]:
len(book_content)

438804

In [23]:
book_content[1000:2000]

'by Mary Wollstonecraft (Godwin) Shelley\n\n\n CONTENTS\n\n Letter 1\n Letter 2\n Letter 3\n Letter 4\n Chapter 1\n Chapter 2\n Chapter 3\n Chapter 4\n Chapter 5\n Chapter 6\n Chapter 7\n Chapter 8\n Chapter 9\n Chapter 10\n Chapter 11\n Chapter 12\n Chapter 13\n Chapter 14\n Chapter 15\n Chapter 16\n Chapter 17\n Chapter 18\n Chapter 19\n Chapter 20\n Chapter 21\n Chapter 22\n Chapter 23\n Chapter 24\n\n\n\n\nLetter 1\n\n_To Mrs. Saville, England._\n\n\nSt. Petersburgh, Dec. 11th, 17—.\n\n\nYou will rejoice to hear that no disaster has accompanied the\ncommencement of an enterprise which you have regarded with such evil\nforebodings. I arrived here yesterday, and my first task is to assure\nmy dear sister of my welfare and increasing confidence in the success\nof my undertaking.\n\nI am already far north of London, and as I walk in the streets of\nPetersburgh, I feel a cold northern breeze play upon my cheeks, which\nbraces my nerves and fills me with delight. Do you understand this\n

## Uncached Request

In [24]:
import time
def make_non_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>"
                },
                {
                    "type": "text",
                    "text": "What happens in chapter 3?"
                }
            ]
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=500,
        messages=messages,
    )
    end_time = time.time()

    return response, end_time - start_time

In [25]:
non_cached_response, non_cached_time = make_non_cached_api_call()
print(f"Non-cached time: {non_cached_time:.2f} seconds")

print("\nOutput (non-cached):")
print(non_cached_response.content)

Non-cached time: 32.13 seconds

Output (non-cached):
[TextBlock(text="In Chapter 3 of Frankenstein, Victor Frankenstein begins his university studies at Ingolstadt. Here are the key events:\n\n1. Victor arrives at the University of Ingolstadt at age 17, feeling lonely and missing his family and home.\n\n2. He meets Professor Krempe, who harshly criticizes Victor's previous studies of alchemists like Cornelius Agrippa, dismissing them as outdated and worthless.\n\n3. Victor then meets Professor Waldman, who has a much more positive influence on him. Unlike Krempe, Waldman respectfully acknowledges the contributions of the ancient alchemists while explaining how modern science has advanced beyond them.\n\n4. Waldman's encouraging and inspiring lecture on chemistry captivates Victor, discussing how modern scientists have gained incredible powers over nature.\n\n5. Victor becomes deeply invested in natural philosophy (science) and chemistry in particular. He impresses his professors with h

In [26]:
non_cached_response.usage

Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=108438, output_tokens=303)

## Cached Version

In [27]:
def make_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>",
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": "What happens in chapter 5?"
                }
            ]
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=500,
        messages=messages,
    )
    end_time = time.time()

    return response, end_time - start_time

In [28]:
response1, duration1 = make_cached_api_call()

In [29]:
response1

Message(id='msg_01KrE7hBXe9KpB3Z5Pdfcsma', content=[TextBlock(text="In Chapter 5 of Frankenstein, Victor Frankenstein finally succeeds in bringing his creation to life. This is the pivotal moment where the creature is first animated. Here are the key events:\n\n1. On a dreary November night, Victor finally completes his work of animating the creature he has assembled from various body parts.\n\n2. When the creature comes to life, Victor is immediately horrified by its appearance, describing its yellow skin, black lips, and watery eyes. He realizes he has created a monster rather than the beautiful being he had imagined.\n\n3. Overwhelmed with disgust and horror, Victor flees from his laboratory and abandons the creature.\n\n4. He retreats to his bedroom but cannot sleep, tormented by nightmares about Elizabeth (his fiancée) turning into the corpse of his dead mother.\n\n5. While in this distressed state, Victor sees the monster at his bedside, reaching out to him and trying to speak. V

In [30]:
response2, duration2 = make_cached_api_call()

RateLimitError: Error code: 429 - {'error': {'message': 'exceeded quota for this month'}}

In [31]:
response2.usage

Usage(cache_creation_input_tokens=0, cache_read_input_tokens=108427, input_tokens=11, output_tokens=320)

In [32]:
duration2

10.1973135471344

## Prompt Caching Pricing

* Cache write tokens are 25% more expensive than base input tokens
* Cache read tokens are 90% cheaper than base input tokens
* Regular input and output tokens are priced at standard rates

## Multi-Turn Caching

In [34]:
messages=[
    # ...long conversation so far
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hello, can you tell me more about the solar system",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    },
    {
        "role": "assistant",
        "content": "Certainly! The solar system is the collection of celestial bodies that orbit our Sun. It consists of eight planets, numerous moons, asteroids, comets, and other objects. The planets, in order from closest to farthest from the Sun, are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Each planet has its own unique characteristics and features. Is there a specific aspect of the solar system you'd like to know more about?"
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Tell me more about Mars.",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    }
]

In [39]:
messages=[
    # ...long conversation so far
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hello, can you tell me more about the solar system",
            }
        ]
    },
    {
        "role": "assistant",
        "content": "Certainly! The solar system is the collection of celestial bodies that orbit our Sun. It consists of eight planets, numerous moons, asteroids, comets, and other objects. The planets, in order from closest to farthest from the Sun, are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Each planet has its own unique characteristics and features. Is there a specific aspect of the solar system you'd like to know more about?"
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Tell me more about Mars.",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    },
    {
        "role": "assistant",
        "content": "I'd love to tell you about Mars.  Mars is...."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "That's really neat.  Tell me about Pluto!",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    },
]